In [ ]:
# General imports.
import ipywidgets as ipw
from IPython.display import display, clear_output

# AiiDA imports.
from aiida.common.exceptions import NotExistent, MultipleObjectsError
from aiida.orm import  load_code

# AiiDAlab imports.
%load_ext aiida
%aiida
import aiidalab_widgets_base as awb

In [ ]:
computational_resources = awb.ComputationalResourcesWidget(
            description="CP2K code:", default_calc_job_plugin="cp2k"
        )

# View plugins

Double-check that the spm (scanning probe microscopy) plugins are installed.

In [ ]:
def on_list_plugins(b):
    with list_plugin_output:
        ! verdi plugin list aiida.calculations

list_plugin_output = ipw.Output()
btn_list_plugins = ipw.Button(description="List all plugins")
btn_list_plugins.on_click(on_list_plugins)
display(btn_list_plugins, list_plugin_output)

# Setup codes

In [ ]:
code_output = ipw.Output()

proxy_cmd = ""
hostname = ""

def show_codes(computer):
    if computer is not None:
        querybuild = QueryBuilder()
        querybuild.append(Computer, tag='computer', filters={'uuid':{'==':computer.uuid}})
        querybuild.append(Code,
                          filters={
                              'extras.hidden': {
                                  "~==": True
                              }
                          },
                          project=['*'],
                          with_computer='computer')
        results = querybuild.all()
        with code_output:
            clear_output()
            print("Codes on " + computer.label + ":")
            for e in results:
                print(e)
                
def update_paths(computer):
    if computer is not None:
        if computer.label == "localhost":
            install_path_inp.value = "/home/jovyan/aiida-soft"
            stm_prepend_text.value =  'eval "$(command conda shell.bash hook 2> /dev/null)"\n'
            stm_prepend_text.value += "conda activate mpi4py-env\n"
            afm_prepend_text.value =  'eval \"$(command conda shell.bash hook 2> /dev/null)"\n'
            afm_prepend_text.value += "conda activate afm-env\n"
        else:
            ssh_config = computer.get_configuration()
            username = 'username'
            if 'username' in ssh_config:
                username = ssh_config['username']
            install_path_inp.value = "/users/%s/aiida-soft" % username
       
    
def validate_computer(computer):
    global proxy_cmd, hostname
    
    computer_ok = False
    localhost = computer.label == "localhost"
    if computer is not None and not localhost:
        
        ssh_config = computer.get_configuration()
        if 'proxy_command' in ssh_config:
            proxy_cmd += f"-o Proxycommand=\"{ssh_config['proxy_command']}\" "
            
        hostname = ""
        if 'username' in ssh_config:
            hostname += f"{ssh_config['username']}@"
        hostname += f"{computer.hostname}"
        
        # check if the ssh connection works
        out = ! ssh {proxy_cmd}{hostname} "echo 1"
        ssh_works = not (len(out) == 0 or out[0] != '1')
        if ssh_works:
            computer_ok = True
        else:
            with code_output:
                print()
                print("Error: SSH connection to the computer is not working properly.")
                print(out)
    
    if localhost:
        computer_ok = True
    if computer_ok:
        retrieve_button.disabled     = False
        stm_setup_btn.disabled       = False
        overlap_setup_btn.disabled   = False
        hrstm_setup_btn.disabled     = False
        afm_retrieve_button.disabled = False
        afm_setup_btn.disabled       = False
    else:
        retrieve_button.disabled     = True
        stm_setup_btn.disabled       = True
        overlap_setup_btn.disabled   = True
        hrstm_setup_btn.disabled     = True
        afm_retrieve_button.disabled = True
        afm_setup_btn.disabled       = True
    

#computer_drop._dropdown.observe(show_codes, names='value')
#computer_drop._dropdown.observe(update_paths, names='value')
#computer_drop._dropdown.observe(validate_computer, names='value')

def on_computer_change(c):
    global computer
    try:
        computer = load_code(c.new).computer
    except:
        computer = None
    show_codes(computer)
    update_paths(computer)
    validate_computer(computer)
computational_resources.observe(on_computer_change,'value')

#show_codes(0)

display(ipw.VBox([computational_resources, code_output]))


Set the path of the remote computer to install the external scripts to. Needs to be a location available to the calculation nodes (e.g. not `/project/` on `daint.cscs.ch`). 

In [ ]:
style = {'description_width': '120px'}
layout = {'width': '70%'}
install_path_inp = ipw.Text(description="Install path:", value="/users/username/aiida-soft", layout=layout, style=style)
display(install_path_inp)

# STM codes

Retrieve the STM python codes to the previously specified install path on the remote machine.

In [ ]:
def retrive_scripts(b):
    
    global version_id, code_dir_path
    
    repository = "https://github.com/nanotech-empa/cp2k-spm-tools"
    
    version = 'master' 
    
    #version = 'develop'
    #version = 'v1.1'
    
    if version == 'master' or version == 'develop':
        commit_hash = ! git ls-remote "{repository}.git" | grep "{version}"
        version_id = commit_hash[0][:6]
        version_no_v = version
    else:
        version_id = version
        version_no_v = version[1:]
    
    code_dir_name = "cp2k-spm-tools-"+version_id

    code_remote_path = install_path_inp.value

    code_dir_path = code_remote_path+"/"+code_dir_name

    #computer = computer_drop.selected_computer
    localhost = computer.label == "localhost"
    with retrieve_output:
        
        clear_output()

        # ssh key needs to be set
        if not localhost:
            ! ssh {proxy_cmd}{hostname} "wget {repository}/archive/{version}.zip -O {code_remote_path}/tmp.zip"
            ! ssh {proxy_cmd}{hostname} "cd {code_remote_path} && unzip -o tmp.zip && rm -rf tmp.zip && mv cp2k-spm-tools-{version_no_v} {code_dir_name}"
            ! ssh {proxy_cmd}{hostname} "cd {code_remote_path}/{code_dir_name} && chmod u+x *"
        else:
            ! wget {repository}/archive/{version}.zip -O {code_remote_path}/tmp.zip
            ! cd {code_remote_path} && unzip -o tmp.zip && rm -rf tmp.zip && mv cp2k-spm-tools-{version_no_v} {code_dir_name}
            ! cd {code_remote_path}/{code_dir_name} && chmod u+x *

code_dir_path = None
version_id = None

retrieve_output = ipw.Output()

retrieve_button = ipw.Button(description="Retrieve scripts", disabled=True)

retrieve_button.on_click(retrive_scripts)

display(retrieve_button, retrieve_output)


For all STM codes, a `python 3` instance with `mpi4py` and `ase` is needed.

Please provide the correct bash commands to make this available. This text will be added each SLURM submission script, just before the execution command. The default value works for `daint.cscs.ch`.

In [ ]:
stm_prepend_text = ipw.Textarea(description="Prepend text:",
             layout = {'width': '70%'},
             value =
                "module load cray-python\n" +
                "export PYTHONPATH=$PYTHONPATH:\"/users/keimre/soft/ase\"\n"
            )

display(stm_prepend_text)

#### Scanning tunnelling microscopy and spectroscopy images

In [ ]:
def setup_stm():
    
    # Eval mol. orbitals
    # Python with MPI support and ASE
    prepend_text  = "### code prepend_text start ###\n"
    prepend_text += stm_prepend_text.value
    prepend_text += "### code prepend_text end ###\n"

    code_label = "py_stm_"+version_id

    code_full_name = code_label + "@" + computer.label

    try:
        Code.get_from_string(code_full_name)
    except (NotExistent, MultipleObjectsError):            
        code = Code(remote_computer_exec=(computer, code_dir_path + "/stm_sts_from_wfn.py"))
        code.label = code_label
        code.description = "Python code to calculate STM and STS"
        code.set_input_plugin_name("spm.stm")
        code.set_prepend_text(prepend_text)
        code.set_append_text("")
        code.store()
        ! verdi code show "{code_full_name}"
        
        #! reentry scan
        ! verdi daemon restart > /dev/null
        
    else:
        print("Code '{}' already existent".format(code_full_name))
        
def on_stm_setup_click(b):
    with stm_setup_out:
        clear_output()
        setup_stm()
    
stm_setup_out = ipw.Output()

stm_setup_btn = ipw.Button(description="Setup stm", disabled=True)
stm_setup_btn.on_click(on_stm_setup_click)

display(stm_setup_btn, stm_setup_out)

#### Orbital overlap

In [ ]:
def setup_overlap():
    
    prepend_text  = "### code prepend_text start ###\n"
    prepend_text += stm_prepend_text.value
    prepend_text += "### code prepend_text end ###\n"

    code_label = "py_overlap_"+version_id

    code_full_name = code_label + "@" + computer.label

    try:
        Code.get_from_string(code_full_name)
    except (NotExistent, MultipleObjectsError):            
        code = Code(remote_computer_exec=(computer, code_dir_path + "/overlap_from_wfns.py"))
        code.label = code_label
        code.description = "Python code to evaluate overlaps between orbitals"
        code.set_input_plugin_name("spm.overlap")
        code.set_prepend_text(prepend_text)
        code.set_append_text("")
        code.store()
        ! verdi code show "{code_full_name}"
        
        #! reentry scan
        ! verdi daemon restart > /dev/null
    else:
        print("Code '{}' already existent".format(code_full_name))
        
def on_overlap_setup_click(b):
    with overlap_setup_out:
        clear_output()
        setup_overlap()
    
overlap_setup_out = ipw.Output()

overlap_setup_btn = ipw.Button(description="Setup overlap", disabled=True)
overlap_setup_btn.on_click(on_overlap_setup_click)

display(overlap_setup_btn, overlap_setup_out)

#### High-Resolution STM

In [ ]:
def setup_hrstm():
    
    # Python with MPI support
    prepend_text  = "### code prepend_text start ###\n"
    prepend_text += stm_prepend_text.value
    prepend_text += "### code prepend_text end ###\n"

    code_label = "py_hrstm_"+version_id

    code_full_name = code_label + "@" + computer.label

    try:
        Code.get_from_string(code_full_name)
    except (NotExistent, MultipleObjectsError):            
        code = Code(remote_computer_exec=(computer, code_dir_path+"/hrstm_from_wfn.py"))
        code.label = code_label
        code.description = "Python code to calculate HR-STM"
        code.set_input_plugin_name("spm.hrstm")
        code.set_prepend_text(prepend_text)
        code.set_append_text("")
        code.store()
        ! verdi code show "{code_full_name}"
        
        #! reentry scan
        ! verdi daemon restart > /dev/null
    else:
        print("Code '{}' already existent".format(code_full_name))
        
def on_hrstm_setup_click(b):
    with hrstm_setup_out:
        clear_output()
        setup_hrstm()
    
hrstm_setup_out = ipw.Output()

hrstm_setup_btn = ipw.Button(description="Setup hrstm", disabled=True)
hrstm_setup_btn.on_click(on_hrstm_setup_click)

display(hrstm_setup_btn, hrstm_setup_out)

# AFM codes

In [ ]:
afm_pp_code_dir_path = None
afm_2pp_code_dir_path = None

# NB: "dev" branch for pp

# initial versions
#sha_pp = "b5f22ca55ea329c8d49aa37338eab0f8f8e21583"
#sha_2pp = "a5f77cc74b238b7ed07ba9a1e8228e70231289de"

# after electrostatics update
#sha_pp = "8aa7a6065f0b99131b313b499105c72584697ad6"
#sha_2pp = "ba8f05088e6745d386798a0587e5e7ab48d810a0"

#def retrive_afm_scripts(b):
#    
#    global afm_pp_code_dir_path, afm_2pp_code_dir_path
#
#    code_dir_name_pp = "ProbeParticleModel_pp"#_%s" % sha_pp[:6]
#    code_dir_name_2pp = "ProbeParticleModel_2pp"#_%s" % sha_2pp[:6]
#
#    code_remote_path = install_path_inp.value
#
#    afm_pp_code_dir_path = code_remote_path+"/"+code_dir_name_pp
#    afm_2pp_code_dir_path = code_remote_path+"/"+code_dir_name_2pp
#
#    #computer = computer_drop.selected_computer
#    localhost = computer.label == "localhost"
#    with afm_retrieve_output:
#        
#        clear_output() 
#
#        if not localhost:
#            # ssh key needs to be set
#            ! ssh {proxy_cmd}{hostname} "wget https://github.com/Probe-Particle/ppafm/archive/refs/heads/main.zip -O {code_remote_path}/{code_dir_name_pp}.zip"
#            ! ssh {proxy_cmd}{hostname} "cd {code_remote_path} && unzip -o {code_dir_name_pp}.zip && rm -rf  {code_dir_name_pp}.zip && mv ppafm-main {code_dir_name_pp} && cp -r ppafm-main {code_dir_name_2pp}"
#            ! ssh {proxy_cmd}{hostname} "cd {afm_2pp_code_dir_path} && git checkout complex_tip"
#            
#            # create a custom bash scripts
#            ! scp {proxy_cmd} "./afm/run_afm.sh" {hostname}":{afm_pp_code_dir_path}/run_afm.sh"
#            ! scp {proxy_cmd} "./afm/run_afm.sh" {hostname}":{afm_2pp_code_dir_path}/run_afm.sh"
#                
#            ! ssh {proxy_cmd}{hostname} "chmod -R 755 {afm_pp_code_dir_path}"
#            ! ssh {proxy_cmd}{hostname} "chmod -R 755 {afm_2pp_code_dir_path}"
#        else:
#            ! wget https://github.com/Probe-Particle/ppafm/archive/refs/heads/main.zip -O {code_remote_path}/{code_dir_name_pp}.zip
#            ! cd {code_remote_path} && unzip -o {code_dir_name_pp}.zip && rm -rf  {code_dir_name_pp}.zip &&  mv ppafm-main {code_dir_name_pp}
#                  
#            ! wget https://github.com/Probe-Particle/ppafm/archive/refs/heads/complex_tip.zip -O {code_remote_path}/{code_dir_name_2pp}.zip
#            ! cd {code_remote_path} && unzip -o {code_dir_name_2pp}.zip && rm -rf  {code_dir_name_2pp}.zip &&  mv ppafm-complex_tip {code_dir_name_2pp} 
#            #&&  mv ppafm-main {code_dir_name_pp}
#
#            # create a custom bash scripts
#            ! cp {proxy_cmd} "./afm/run_afm.sh" {afm_pp_code_dir_path}/run_afm.sh
#            ! cp {proxy_cmd} "./afm/run_afm.sh" {afm_2pp_code_dir_path}/run_afm.sh
#            ! chmod -R 755 {afm_pp_code_dir_path}
#            ! chmod -R 755 {afm_2pp_code_dir_path}
#
#afm_retrieve_output = ipw.Output()
#
#afm_retrieve_button = ipw.Button(description="Retrieve scripts", disabled=True)
#
#afm_retrieve_button.on_click(retrive_afm_scripts)
#
#display(afm_retrieve_button, afm_retrieve_output)


For the AFM codes, a python instance with `matplotlib` is needed.

Please provide the correct bash commands to make this available (this text will be prepended to each slurm script).

In [ ]:
afm_prepend_text = ipw.Textarea(description="Prepend text:",
             layout = {'width': '70%'},
             value =
                "export PATH='/users/keimre/soft/miniconda3/envs/py27/bin':$PATH\n"
            )

display(afm_prepend_text)

In [ ]:
def setup_afm_code(label, code_dir_path, description):
    
    prepend_text  = "### code prepend_text start ###\n"
    prepend_text += afm_prepend_text.value
    prepend_text += "### code prepend_text end ###\n"

    code_label = label

    code_full_name = code_label + "@" + computer.label

    try:
        Code.get_from_string(code_full_name)
    except (NotExistent, MultipleObjectsError):            
        code = Code(remote_computer_exec=(computer, code_dir_path + "/run_afm.sh"))
        code.label = code_label
        code.description = description
        code.set_input_plugin_name("spm.afm")
        code.set_prepend_text(prepend_text)
        code.set_append_text("")
        code.store()
        ! verdi code show "{code_full_name}"
        
        #! reentry scan
        ! verdi daemon restart > /dev/null
    else:
        print("Code '{}' already existent".format(code_full_name))
        
def on_afm_setup_click(b):
    with afm_setup_out:
        clear_output()
        setup_afm_code("py_afm_pp_", afm_pp_code_dir_path, "Python code for ProbeParticle AFM simulation")
        setup_afm_code("py_afm_2pp_", afm_2pp_code_dir_path, "Python code for 2 point ProbeParticle AFM simulation")
    
afm_setup_out = ipw.Output()

afm_setup_btn = ipw.Button(description="Setup afm codes", disabled=True)
afm_setup_btn.on_click(on_afm_setup_click)

display(afm_setup_btn, afm_setup_out)